# 데이터 전처리 - binning
* [Data binning - Wikipedia](https://en.wikipedia.org/wiki/Data_binning)
* 수치형 데이터를 카테고리 형태로 바꾸는 것
* 사소한 관측 오류의 영향을 줄이기 위해 사용되는 데이터 전처리 기술
* 속성 값들을 구간(bin)별로 나누고 구간별 평균 또는 구간 중앙값으로 바꿔 바이너리화(이산화)하는 것
* 지정된 빈(bin)의 개수에 기반한 하향식(Top-Down) 분할 기법
* 통계적 데이터 비닝 은 더 많거나 적은 연속 값을 더 작은 수의 "bin"으로 그룹화하는 방법이다. 예를 들어, 사용자 그룹에 대한 데이터가있는 경우, 더 적은 수의 연령 간격으로 연령을 정렬 할 수 있다.
* 반대로 카테고리 데이터를 바이너리 수치형 데이터로 바꾸는 것을 인코딩이라고 한다.


## 버케팅(bucketing)
하나의 특성(일반적으로 연속)을 버킷(bucket) 또는 빈(bin)이라고 하는 **여러 이진 특성으로 변환하는 작업**으로서, 일반적으로 값 범위를 기준으로 합니다. 예를 들어 온도를 하나의 부동 소수점 연속 특성으로 표현하는 대신 온도 범위를 불연속 빈으로 나눌 수 있습니다. 민감도가 1/10도인 온도 데이터가 있다면 0.0~15.0도 범위의 모든 온도를 1번 빈에, 15.1~30.0도 범위를 2번 빈에, 30.1~50.0도 범위를 3번 빈에 넣을 수 있습니다.(출처 : [Machine Learning Glossary - Google Developers](https://developers.google.com/machine-learning/glossary/#bucketing))



In [1]:
import pandas as pd
import numpy as np
import missingno as msno # 누락 된 데이터를 시각화 해주기 위해

import sys
print(sys.version)
print(pd.__version__)
print(np.__version__)
print(msno.__version__)

3.6.1 (v3.6.1:69c0db5050, Mar 21 2017, 01:21:04) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]
0.21.0
1.14.0
0.3.5


In [2]:
# 판다스를 통해 데이터를 로드해 온다.
# 여기에서는 캐글의 타이타닉 데이터를 사용한다. 
# 데이터 다운로드 : https://www.kaggle.com/c/titanic/data 
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

print(train.shape)
print(test.shape)

(891, 12)
(418, 11)


## 값의 기준을 정해주고 채워주기

In [3]:
# 티켓 요금에 따라 Low, Medium, High로 분류
train["Fare_Low"] = train["Fare"] < 10
train["Fare_Medium"] = (train["Fare"] >= 10) & (train["Fare"] < 50 )
train["Fare_High"] = train["Fare"] >= 50

## pandas의 cut, qcut을 이용
* [pandas.qcut — pandas 0.22.0 documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.qcut.html)
* bin의 size를 정해주거나, 

In [4]:
# binning 전에 결측치를 채워준다.
# 중간값으로 누락된 요금을 채워준다.
train['Fare'].fillna(train['Fare'].median(), inplace = True)
# 중간값으로 누락된 나이 데이터를 채워준다.
train['Age'].fillna(train['Age'].median(), inplace = True)

In [5]:
# Continuous variable bins; qcut vs cut: https://stackoverflow.com/questions/30211923/what-is-the-difference-between-pandas-qcut-and-pandas-cut
# Fare Bins/Buckets using qcut or frequency bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.qcut.html
# 몇 개 단위의 Bin으로 묶어줄지 지정해 주면 알아서 데이터를 분리해 줌
# qcut은 동일한 갯수로 bin에 들어가도록 담아줌
train['FareBin'] = pd.qcut(train['Fare'], 4)

# Age Bins/Buckets using cut or value bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.cut.html
# cut은 동일한 간격으로 나눠줌
train['AgeBin'] = pd.cut(train['Age'].astype(int), 5)

In [6]:
train[['Fare', 'FareBin', 'Age', 'AgeBin']].head()

,Fare,FareBin,Age,AgeBin
0,7.2500,"(-0.001, 7.91]",22.0,"(16.0, 32.0]"
1,71.2833,"(31.0, 512.329]",38.0,"(32.0, 48.0]"
2,7.9250,"(7.91, 14.454]",26.0,"(16.0, 32.0]"
3,53.1000,"(31.0, 512.329]",35.0,"(32.0, 48.0]"
4,8.0500,"(7.91, 14.454]",35.0,"(32.0, 48.0]"


In [7]:
train['FareBin'].value_counts()

(7.91, 14.454]     224
(-0.001, 7.91]     223
(31.0, 512.329]    222
(14.454, 31.0]     222
Name: FareBin, dtype: int64

In [8]:
train['AgeBin'].value_counts()

(16.0, 32.0]     525
(32.0, 48.0]     186
(-0.08, 16.0]    100
(48.0, 64.0]      69
(64.0, 80.0]      11
Name: AgeBin, dtype: int64

In [9]:
train['FamilySize'] = train['SibSp'] + train['Parch']+1
test['FamilySize'] = test['SibSp'] + test['Parch']+1
print(train['FamilySize'].value_counts())

1     537
2     161
3     102
4      29
6      22
5      15
7      12
11      7
8       6
Name: FamilySize, dtype: int64


In [10]:
train.loc[train['FamilySize'] == 1, 'FsizeD'] = 'singleton'
train.loc[(train['FamilySize'] > 1)  &  (train['FamilySize'] < 5) , 'FsizeD'] = 'small'
train.loc[train['FamilySize'] >4, 'FsizeD'] = 'large'

test.loc[test['FamilySize'] == 1, 'FsizeD'] = 'singleton'
test.loc[(test['FamilySize'] >1) & (test['FamilySize'] <5) , 'FsizeD'] = 'small'
test.loc[test['FamilySize'] >4, 'FsizeD'] = 'large'
print(train['FsizeD'].unique())
print(train['FsizeD'].value_counts())

['small' 'singleton' 'large']
singleton    537
small        292
large         62
Name: FsizeD, dtype: int64


In [11]:
train['FamilySize'].value_counts()

1     537
2     161
3     102
4      29
6      22
5      15
7      12
11      7
8       6
Name: FamilySize, dtype: int64

In [12]:
train['FamilyBin'] = pd.cut(train['FamilySize'], [0, 1, 4, 11], labels=['S', 'M', 'L'])
train[['FamilySize', 'FamilyBin']].head()

,FamilySize,FamilyBin
0,2,M
1,2,M
2,1,S
3,2,M
4,1,S


In [13]:
train['FamilyBin'].value_counts()

S    537
M    292
L     62
Name: FamilyBin, dtype: int64